In [1]:
import numpy as np

In [2]:
P_transition = np.array([[0.75, 0.15, 0.10],
                         [0.25, 0.55, 0.20],
                         [0.30, 0.30, 0.40]])
P_emission = np.array([[0.75, 0.15, 0.65],
                       [0.25, 0.85, 0.35]])
P_init = [0.65, 0.20, 0.15]

In [3]:
T = 3
hidden_states = np.zeros((T,), dtype=np.int32)
probs = np.zeros((T, 3))

In [4]:
for t in range(T):
    if t == 0:
        probs[t, :] = P_init * P_transition[1, :]
    else:
        probs[t, :] = np.max(probs[t-1, :, None] * P_transition, axis=0)
    hidden_states[t] = np.argmax(probs[t, :])
state_names = ['Sunny', 'Rainy', 'Foggy']
forecast = [state_names[s] for s in hidden_states]
print(forecast)

['Sunny', 'Sunny', 'Sunny']


In [5]:
P_transition = np.array([[0.75, 0.15, 0.10],
                         [0.25, 0.55, 0.20],
                         [0.30, 0.30, 0.40]])

P_emission = np.array([[0.75, 0.15, 0.65],
                       [0.25, 0.85, 0.35],
                       [0.0, 0.0, 0.0]])
P_init = np.array([0.65, 0.20, 0.15])
P_hidden_init = P_transition[1, :]
alpha = np.zeros((3,))
for i in range(3):
    alpha[i] = P_emission[i, 0] * P_init[i]
for t in range(1, 3):
    alpha = np.dot(alpha, P_transition) * P_emission[:, t]
P_evidence = np.sum(alpha)
posteriors = alpha / P_evidence
state_names = ['Sunny', 'Rainy', 'Foggy']
for i in range(3):
    print("Probability of being in state %s: %.4f" % (state_names[i], posteriors[i]))
most_likely_weather = state_names[np.argmax(posteriors)]
print("The most likely weather is %s." % most_likely_weather)

Probability of being in state Sunny: 0.6812
Probability of being in state Rainy: 0.3188
Probability of being in state Foggy: 0.0000
The most likely weather is Sunny.


In [6]:
E = ["no umbrella", "umbrella", "umbrella", "no umbrella", "umbrella", "no umbrella"]
alpha = np.zeros((3,))
for i in range(3):
    alpha[i] = P_emission[i, 0] * P_init[i]
for t in range(1, len(E)):
    alpha = np.dot(alpha, P_transition) * P_emission[:, t % 2]
final_state = np.argmax(alpha)
state_names = ['Sunny', 'Rainy', 'Foggy']
most_likely_weather = state_names[final_state]
print("Final state:", state_names[final_state])
print("Most likely weather:", most_likely_weather)


Final state: Rainy
Most likely weather: Rainy


In [7]:
P_emission = np.array([[0.6, 0.4, 0.0, 0.0],
                       [0.0, 0.0, 0.5, 0.5],
                       [0.3, 0.3, 0.2, 0.2]])
P_transition = np.array([[0.7, 0.2, 0.1],
                         [0.2, 0.5, 0.3],
                         [0.3, 0.3, 0.4]])

E = [0, 1, 2, 2, 1, 0]
T = len(E)
trellis = np.zeros((T, 3))
backpointers = np.zeros((T-1, 3), dtype=np.int64)
for i in range(3):
    trellis[0, i] = P_emission[i, E[0]] * P_transition[i, 0]
for t in range(1, T):
    for j in range(3):
        prob_transitions = trellis[t - 1, :] * P_transition[:, j]
        trellis[t, j] = P_emission[j, E[t]] * np.max(prob_transitions)
        backpointers[t - 1, j] = np.argmax(prob_transitions)
hidden_states = [np.argmax(trellis[-1])]
for i in range(T-2, -1, -1):
    hidden_states.append(backpointers[i, hidden_states[-1]])
hidden_states.reverse()
print("The most likely sequence of hidden states is:", hidden_states)

The most likely sequence of hidden states is: [0, 0, 1, 1, 0, 0]
